In [2]:
import polars as pl 


df = pl.read_csv("Dmel.iorfs", new_columns = ["length"])

1st loop:   0%|          | 0/4 [00:00<?, ?it/s]

2nd loop:   0%|          | 0/5 [00:00<?, ?it/s]

3rd loop:   0%|          | 0/50 [00:00<?, ?it/s]

3rd loop:   0%|          | 0/50 [00:00<?, ?it/s]

3rd loop:   0%|          | 0/50 [00:00<?, ?it/s]

...

3rd loop:   0%|          | 0/50 [00:00<?, ?it/s]

3rd loop:   0%|          | 0/50 [00:00<?, ?it/s]

3rd loop:   0%|          | 0/50 [00:00<?, ?it/s]

3rd loop:   0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
print(multiprocessing.cpu_count())  

4
